<a href="https://colab.research.google.com/github/modhudeb/Spam-Detection-Bidirectional-GRU/blob/main/Spam_detection_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [27]:
import numpy as np
import pandas as pd
import tensorflow as tf
import gensim

In [ ]:
!pip install contractions

In [29]:
import re
import nltk
import contractions

In [ ]:
nltk.download('wordnet')

In [31]:
# !pip install transformers

In [ ]:
!pip install nlpaug

In [33]:
import nlpaug.augmenter.word as naw

In [134]:
import tensorflow.keras as keras
from sklearn.metrics import classification_report

In [34]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

In [132]:
from keras.callbacks import EarlyStopping

# Loading data

In [ ]:
data = pd.read_csv('emails.csv', encoding='latin-1')

In [ ]:
df = data.copy()

# Code

## Preprocessing part-1 (collecting data from different datasets and merge)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5728 entries, 0 to 5727
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    5728 non-null   object
 1   Spam    5728 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 89.6+ KB


In [ ]:
df.iloc[0]

Text    Subject: naturally irresistible your corporate...
Spam                                                    1
Name: 0, dtype: object

In [ ]:
spams = df[df['Spam']==1]

In [ ]:
spams.head(2)

,Text,Spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1


In [ ]:
string = "Subject: naturally            irresistible your corporate identity lt is really hard to recollect a company : the market is full of suqgestions and the information isoverwhelminq ; but a good catchy logo , stylish statlonery and outstanding website will make the task much easier . we do not promise that havinq ordered a iogo your company will automaticaily become a world ieader : it isguite ciear that without good products , effective business organization and practicable aim it will be hotat nowadays market ; but we do promise that your marketing efforts will become much more effective . here is the list of clear benefits : creativeness : hand - made , original logos , specially done to reflect your distinctive company image . convenience : logo and stationery are provided in all formats ; easy - to - use content management system letsyou change your website content and even its structure . promptness : you will see logo drafts within three business days . affordability : your marketing break - through shouldn ' t make gaps in your budget . 100 % satisfaction guaranteed : we provide unlimited amount of changes with no extra fees for you to be surethat you will love the result of this collaboration . have a look at our portfolio _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ not interested . . . _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _"

In [ ]:
def cleanString(X:str):
  X = X.replace('Subject: ', '')
  X = re.sub(r'[^a-zA-Z.,\'\s]', '', X)
  X = re.sub(r'\s+', ' ', X.strip())
  return X

In [ ]:
cleanString(string)

"naturally irresistible your corporate identity lt is really hard to recollect a company the market is full of suqgestions and the information isoverwhelminq but a good catchy logo , stylish statlonery and outstanding website will make the task much easier . we do not promise that havinq ordered a iogo your company will automaticaily become a world ieader it isguite ciear that without good products , effective business organization and practicable aim it will be hotat nowadays market but we do promise that your marketing efforts will become much more effective . here is the list of clear benefits creativeness hand made , original logos , specially done to reflect your distinctive company image . convenience logo and stationery are provided in all formats easy to use content management system letsyou change your website content and even its structure . promptness you will see logo drafts within three business days . affordability your marketing break through shouldn ' t make gaps in you

In [ ]:
spams['Text'] = spams['Text'].apply(cleanString)

<ipython-input-98-d1db41506c02>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spams['Text'] = spams['Text'].apply(cleanString)


Loading another DataFrame

In [ ]:
df = pd.read_csv('SMSCollection.csv')

In [ ]:
df.head(2)

,Class,sms
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...


In [ ]:
df['Class'].value_counts()

ham     4825
spam     747
Name: Class, dtype: int64

In [ ]:
for x in spams['Text']:
  df.loc[len(df)] = ['spam', x]

print('Done')

Done


In [ ]:
df['Class'].value_counts()

ham     4825
spam    2115
Name: Class, dtype: int64

In [ ]:
df.to_csv('SpamHamDataset.csv', index=False)

## Preprocessing part-2

In [36]:
data = pd.read_csv('SpamHamDataset.csv')

In [37]:
df = data.copy()

In [38]:
df.head()

,Class,sms
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [39]:
df.columns = ['Spam', 'Text']

In [40]:
df['Spam'] = df['Spam'].apply(lambda x : 0 if x == 'ham' else 1)

In [41]:
df['Spam'].value_counts()

0    4825
1    2115
Name: Spam, dtype: int64

In [42]:
spamsDf = df['Text'][df['Spam'] == 1].reset_index(drop=True)
hamsDf = df['Text'][df['Spam'] == 0].reset_index(drop=True)

In [43]:
spamsDf.tail()

2110    are you ready to get it hello viagra is the me...
2111    would you like a gas card don ' t let the curr...
2112    immediate reply needed dear sir , i am dr jame...
2113    wanna see me get fisted fist bang will show yo...
2114    hot stock info drgv announces another press re...
Name: Text, dtype: object

In [44]:
hamsDf.loc[len(hamsDf)] = 'Hey how are you dear ?'

In [45]:
hamsDf.loc[len(hamsDf)-1]

'Hey how are you dear ?'

### Lets handle this imbalanced text data with augmentation

We will augment spamsDf to balance the data

In [46]:
def augmentWord(text, n=2):
  augmenter = naw.SynonymAug(aug_src = 'wordnet', aug_min=3, aug_max = 20, aug_p = 0.2)
  aug_text = augmenter.augment(text, n)

  return aug_text

In [47]:
augmentWord("Hey there are many illegal videos available for free",3)

['Hey there be many illegal videos available for gratuitous',
 'Hey there embody many illegal videos available for free',
 'Hey there are many illegal tv available for free']

In [48]:
import copy

In [49]:
count = 0
spamlen = copy.copy(len(spamsDf))
for i in range(spamlen-1):   # we can't write the len(spamsDf) directly...cause afterwards the spamsDf will be updated and the len into range() will keep changing
  # This `if condition` part is for dividing longer texts into parts..
  length = len(spamsDf[i].split(' '))       # count words in a sentence
  if length > 200:
    #
    temp = spamsDf[i].split(".")
    tlen = len(temp)
    #
    if length >= 2000:
      try :
        pct = 5      # 5% of temp variable's length
        st = 0
        for _ in range(7):
          if pct <= 40 :
            aug = augmentWord(". ".join(temp[st : int(tlen*(pct/100))]),n=2)
            spamsDf[len(spamsDf)] = aug[0]
            spamsDf[len(spamsDf)] = aug[1]
            st = int(tlen*(pct/100))
            pct += 5
      except Exception as e:
        count += 1
        print("Error index : ", i)
    #
    elif length > 500:
      try :
        pct = 9      # 9% of temp variable's length
        st = 0
        for j in range(10):
          if pct <= 100 :
            aug = augmentWord(". ".join(temp[st : int(tlen*(pct/100))]),n=3)
            spamsDf[len(spamsDf)] = aug[0]
            spamsDf[len(spamsDf)] = aug[1]
            spamsDf[len(spamsDf)] = aug[2]
            st = int(tlen*(pct/100))
            pct += 10
      except Exception as e:
        count += 1
        print("ERROR index : ", i)
    #
    else:
      try :
        pct = 30      # 30% of temp variable's length
        st = 0
        for k in range(3):
          if pct <= 99 :
            aug = augmentWord(". ".join(temp[st : int(tlen*(pct/100))]), n=3)
            spamsDf[len(spamsDf)] = aug[0]
            spamsDf[len(spamsDf)] = aug[1]
            spamsDf[len(spamsDf)] = aug[2]
            # spamsDf[len(spamsDf)] = aug[3]
            st = int(tlen*(pct/100))
            pct += 30
      except Exception as e:
        count += 1
        print("Error index : ", i)
    #
    spamsDf.drop(i, inplace = True)

  else:
    aug = augmentWord(spamsDf[i], n = 4)
    spamsDf[len(spamsDf)] = aug[0]
    spamsDf[len(spamsDf)] = aug[1]
    spamsDf[len(spamsDf)] = aug[2]
    spamsDf[len(spamsDf)] = aug[3]

spamsDf.reset_index(drop = True, inplace = True)
del(i, j, _, temp, aug)    # to save the ram memory
print(r'Total spam texts : {}'.format(len(spamsDf)))

ERROR index :  1610
Error index :  1826
Total spam texts : 4743


In [50]:
len(spamsDf) - len(hamsDf)

-83

The output says DataSet is almost balanced

In [51]:
# removing sentences that contain 200+ word
count = 0
for x,y in enumerate(spamsDf):
# for y in spamsDf:
  if len(y.split(' ')) > 200:
    spamsDf.drop(x, inplace = True)
    count+=1
print(count)

10


In [52]:
# count = 0
# xx = []
# for x,y in enumerate(spamsDf):
# # for y in spamsDf:
#   if len(y.split(' ')) > 200:
#     count+=1
#     xx.append(x)
# print(xx)
# print(count)

we will concate these series together. to make a new balanced dataset

In [53]:
hamLabel = np.zeros((len(hamsDf),))
spamLabel = np.ones((len(spamsDf),))

In [54]:
len(hamLabel)

4826

In [55]:
labels = np.concatenate((hamLabel, spamLabel), axis = 0)
len(labels)

9559

In [64]:
texts = pd.concat([hamsDf, spamsDf], ignore_index=True)
len(texts)

9559

### Text cleaning

In [66]:
stop_words = ['a', 'an', 'the', 'is', 'am', 'are', 'was', 'were', 'have', 'has', 'had', 'might', 'must','and', 'or', 'either', 'neither']

In [67]:
def clean_text_tokenize(X):
  lem = WordNetLemmatizer()
  X = contractions.fix(X.lower())
  X = re.sub(r'[^a-zA-Z0-9\s]', '', X)
  X = word_tokenize(X)
  X = [lem.lemmatize(word) for word in X if word not in stop_words]
  return X



In [68]:
clean_text_tokenize("You're a really jerking ?, and have good 69 knowledges.")

['you', 'really', 'jerking', 'good', '69', 'knowledge']

In [69]:
texts.head(2)

0    Go until jurong point, crazy.. Available only ...
1                        Ok lar... Joking wif u oni...
Name: Text, dtype: object

In [70]:
texts = texts.apply(clean_text_tokenize)
texts.head(2)

0    [go, until, jurong, point, crazy, available, o...
1                     [ok, lar, joking, wif, you, oni]
Name: Text, dtype: object

## Word2Vec Embedding

We will vectorize these tokenz using Word2Vec

In [ ]:
w2v = gensim.models.Word2Vec(window=3, min_count = 1, vector_size=64)
w2v.build_vocab(texts)

In [ ]:
w2v.corpus_count

In [ ]:
w2v.train(texts, epochs = 10, total_examples= w2v.corpus_count)

In [ ]:
w2v.wv.most_similar('sex')

In [ ]:
# w2v.save('wordVecModel.bin')

In [ ]:
# del(w2v)

In [12]:
model_wv = gensim.models.Word2Vec.load('wordVecModel.bin')

We are done embedding. We will assign every word with its corresponding vector (for every sentence)

In [ ]:
texts.head(2)

0    [go, until, jurong, point, crazy, available, o...
1                     [ok, lar, joking, wif, you, oni]
Name: Text, dtype: object

## GRU - USING TENSORFLOW EMBEDDING

In [135]:
## we need padded sequence of these sentence vectors (as there are not all equal number of words in each sentence )
wvDict = model_wv.wv.key_to_index
def wordIndexing(X:list):
  X = [wvDict[word] for word in X if word in wvDict.keys()]
  return X


texts_vectored = texts.apply(wordIndexing)
max_len = max([len(sent) for sent in texts_vectored])
print(max_len)
texts_vectored_padded = tf.keras.preprocessing.sequence.pad_sequences(texts_vectored, maxlen= max_len, padding = 'post')

179


#### We will train the model now

In [137]:
from sklearn.model_selection import train_test_split

In [138]:
xtrain, xtest, ytrain, ytest = train_test_split(texts_vectored_padded, labels, test_size=0.2, random_state = 42, stratify=labels)

In [139]:
# del(texts_vectored, texts_vectored_padded) # ram saving

In [140]:
xtrain.shape

(7647, 179)

In [141]:
xtest.shape

(1912, 179)

In [ ]:
# GRU MODEL HERE

In [142]:
early_stopping = EarlyStopping(monitor='loss', patience=3)

In [144]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(len(wvDict.keys()), 64, input_length=max_len),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = 'Accuracy')
# but 'adadelta' optimizer is best for text classification
model.fit(xtrain, ytrain, epochs = 100, batch_size=32, callbacks=early_stopping)

Epoch 1/100
239/239 [==============================] - 46s 172ms/step - loss: 0.1957 - Accuracy: 0.9116
Epoch 2/100
239/239 [==============================] - 18s 76ms/step - loss: 0.0198 - Accuracy: 0.9940
Epoch 3/100
239/239 [==============================] - 10s 43ms/step - loss: 0.0046 - Accuracy: 0.9990
Epoch 4/100
239/239 [==============================] - 7s 30ms/step - loss: 0.0017 - Accuracy: 0.9999
Epoch 5/100
239/239 [==============================] - 6s 26ms/step - loss: 0.0284 - Accuracy: 0.9969
Epoch 6/100
239/239 [==============================] - 6s 23ms/step - loss: 0.0013 - Accuracy: 0.9999
Epoch 7/100
239/239 [==============================] - 7s 28ms/step - loss: 7.4189e-04 - Accuracy: 0.9999
Epoch 8/100
239/239 [==============================] - 6s 27ms/step - loss: 3.8282e-04 - Accuracy: 0.9999
Epoch 9/100
239/239 [==============================] - 5s 22ms/step - loss: 2.3863e-04 - Accuracy: 1.0000
Epoch 10/100
239/239 [==============================] - 5s 21ms/st

In [145]:
ypred = model.predict(xtest)

60/60 [==============================] - 1s 7ms/step


In [146]:
ypred = np.round(ypred)

In [147]:
from sklearn.metrics import classification_report
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99       965
         1.0       0.99      0.99      0.99       947

    accuracy                           0.99      1912
   macro avg       0.99      0.99      0.99      1912
weighted avg       0.99      0.99      0.99      1912



In [ ]:
# model.save('SpamDetectModel.h5')

In [148]:
# Pipeline system
print("Build config :\n", model.get_build_config()['input_shape'])
maxlen = model.get_build_config()['input_shape'][1]

Build config :
 (None, 179)


In [149]:
def clean_text_tokenize(X:str) -> list:
  """
  Return:
  ------
  ``1D array of cleaned text with tokenization``
  >>> ['sourav', 'busy', 'now]
  """
  stop_words = ['a', 'an', 'the', 'is', 'am', 'are', 'was', 'were', 'have', 'has', 'had', 'might', 'must','and', 'or', 'either', 'neither', 'what', 'when','how', 'where', 'which']
  lem = WordNetLemmatizer()
  X = contractions.fix(X.lower())
  X = re.sub(r'[^a-zA-Z0-9\s]', '', X)
  X = word_tokenize(X)
  X = [lem.lemmatize(word) for word in X if word not in stop_words]
  return X


def testSpam(X:str):
  X = clean_text_tokenize(X)
  if len(X) > maxlen:
    X = X[0:maxlen]
  # vectorize input text and padding
  corpus = model_wv.wv.key_to_index
  # X = [model_wv.wv[word] for word in X if word in corpus]
  X = [corpus[word] for word in X if word in corpus.keys()]
  # X = keras.preprocessing.sequence.pad_sequences([X], maxlen = maxlen, padding = 'post')
  X = keras.preprocessing.sequence.pad_sequences([X], maxlen= maxlen, padding = 'post')
  X = model.predict(X)
  # return np.round(X)
  return X


In [ ]:
df[df['Spam']==1].sample(5)

In [153]:
testSpam("Send SMS SAX to 7789 to claim your prize money")*100

1/1 [==============================] - 0s 22ms/step


array([[99.89655]], dtype=float32)

In [152]:
testSpam("Congratulations!! You have won the VEGAS SINS LOTTERY and have received $5000 prize. To receive the money contact us")*100

1/1 [==============================] - 0s 31ms/step


array([[99.997734]], dtype=float32)

In [158]:
testSpam("Change your brand names to generics , from overexpenditures to great savings with a tight budget ")*100

1/1 [==============================] - 0s 34ms/step


array([[61.17826]], dtype=float32)

In [163]:
testSpam("Hey, actually my ex awants to come back nut i am confused bout getting back. I will need some money")*100

1/1 [==============================] - 0s 40ms/step


array([[0.00883873]], dtype=float32)

In [160]:
model.save('3_SpamDetectModel.h5')

## GRU - USING Word2Vec embedding

In [71]:
def wordVectoring(X:list):
  try:
    tmp = []
    if len(X) != 0:
      for word in X:
        if word in model_wv.wv.index_to_key:
          tmp.append(model_wv.wv[word])
      return np.stack(tmp)
    else :
      return np.stack([[0]*64])
  except Exception :
    print(Exception)



In [72]:
texts_vectored = texts.apply(wordVectoring)

In [73]:
maxlen = max([len(sent) for sent in texts_vectored])
print("Max length : " ,maxlen)

inShape = (maxlen, model_wv.vector_size)
print("Input shape : " , inShape)

Max length :  179
Input shape :  (179, 64)


In [74]:
def do_pad_sequence(X:np.ndarray, maxlen:int, fillvalue:int = 0):
  return np.array([np.concatenate([sent, np.full(((maxlen - sent.shape[0]), sent.shape[1]), fillvalue)]) for sent in X])


In [75]:
# lets apply the function to get padded sequence
texts_vectored_padded = do_pad_sequence(texts_vectored, maxlen=maxlen)
# texts_vectored_padded = keras.preprocessing.sequence.pad_sequences(texts_vectored, maxlen = maxlen, padding ='post')

In [76]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(texts_vectored_padded, labels, test_size=0.2, random_state = 42, stratify=labels)

In [77]:
# to save ram run this
# del(texts, spamsDf, hamsDf, texts_vectored, texts_vectored_padded)

In [94]:
early_stopping = EarlyStopping(monitor='loss', patience=3)

In [120]:
model2 = tf.keras.models.Sequential([
    tf.keras.layers.GRU(64, input_shape = inShape),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model2.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['Accuracy'])
model2.fit(x_train, y_train, epochs = 70, batch_size=16, callbacks=early_stopping)

Epoch 1/70
478/478 [==============================] - 6s 8ms/step - loss: 0.6936 - Accuracy: 0.5026
Epoch 2/70
478/478 [==============================] - 4s 9ms/step - loss: 0.6934 - Accuracy: 0.5012
Epoch 3/70
478/478 [==============================] - 4s 8ms/step - loss: 0.5751 - Accuracy: 0.6274
Epoch 4/70
478/478 [==============================] - 4s 8ms/step - loss: 0.1556 - Accuracy: 0.9545
Epoch 5/70
478/478 [==============================] - 4s 9ms/step - loss: 0.1204 - Accuracy: 0.9629
Epoch 6/70
478/478 [==============================] - 4s 8ms/step - loss: 0.1031 - Accuracy: 0.9693
Epoch 7/70
478/478 [==============================] - 6s 13ms/step - loss: 0.0898 - Accuracy: 0.9724
Epoch 8/70
478/478 [==============================] - 6s 12ms/step - loss: 0.0790 - Accuracy: 0.9762
Epoch 9/70
478/478 [==============================] - 4s 8ms/step - loss: 0.0692 - Accuracy: 0.9817
Epoch 10/70
478/478 [==============================] - 4s 9ms/step - loss: 0.0628 - Accuracy: 0.98

In [121]:
from sklearn.metrics import classification_report

ypred = model2.predict(x_test)
print(classification_report(y_test, np.round(ypred)))

60/60 [==============================] - 1s 5ms/step
              precision    recall  f1-score   support

         0.0       0.98      0.98      0.98       965
         1.0       0.98      0.97      0.98       947

    accuracy                           0.98      1912
   macro avg       0.98      0.98      0.98      1912
weighted avg       0.98      0.98      0.98      1912



In [82]:
# Pipeline-type-function

In [122]:
model2.get_build_config()['input_shape']

(None, 179, 64)

In [165]:
# From The input shape
maxlen = model2.get_build_config()['input_shape'][1]

In [124]:
def clean_text_tokenize(X:str) -> list:
  """
  Return:
  ------
  ``1D array of cleaned text with tokenization``
  >>> ['sourav', 'busy', 'now]
  """
  stop_words = ['a', 'an', 'the', 'is', 'am', 'are', 'was', 'were', 'have', 'has', 'had', 'might', 'must','and', 'or', 'either', 'neither', 'what', 'when','how', 'where', 'which']
  lem = WordNetLemmatizer()
  X = contractions.fix(X.lower())
  X = re.sub(r'[^a-zA-Z0-9\s]', '', X)
  X = word_tokenize(X)
  X = [lem.lemmatize(word) for word in X if word not in stop_words]
  return X



def update_dictionary(X:list)-> None:
  """
  `X : Need 2D list arrray as input`
  """
  model_wv.build_vocab(X, update=True)
  model_wv.train(X, total_examples=model_wv.corpus_count, epochs = 10)


def testSpam_w(X:str):
  X = clean_text_tokenize(X)
  if len(X) > maxlen:
    X = X[0:maxlen]
  #
  # vectorize input text and padding
  corpus = model_wv.wv.index_to_key
  X = [model_wv.wv[word] for word in X if word in corpus]
  X = keras.preprocessing.sequence.pad_sequences([X], maxlen = maxlen, padding = 'post')
  X = model2.predict(X)
  # return np.round(X)
  return X


In [86]:
df[df['Spam']==1].head()

,Spam,Text
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
5,1,FreeMsg Hey there darling it's been 3 week's n...
8,1,WINNER!! As a valued network customer you have...
9,1,Had your mobile 11 months or more? U R entitle...
11,1,"SIX chances to win CASH! From 100 to 20,000 po..."


In [127]:
testSpam_w("Congratulations!! You have won the VEGAS SINS LOTTERY. Please collect your rewards prize 5000 from us. Email at sourav@gmail.com ")*100

1/1 [==============================] - 0s 21ms/step


array([[98.17656]], dtype=float32)

In [130]:
testSpam_w("only 3 chance to win a bet ! Hit the bell and check your luck . Great chance to win $50000 and receive your prize directly to your bank account")*100

1/1 [==============================] - 0s 99ms/step


array([[99.71419]], dtype=float32)

In [164]:
testSpam("Hey, actually my ex awants to come back nut i am confused bout getting back. I will need some money")*100

1/1 [==============================] - 0s 62ms/step


array([[0.00883873]], dtype=float32)

In [131]:
model2.save('2_SpamDetectModel.h5')